In [57]:
'''
Project scenario (CourseEra practice project):
An international firm that is looking to expand its business in different countries across the world has recruited you. 
You have been hired as a junior Data Engineer and are tasked with creating an automated script that can extract the list of all countries in order of 
their GDPs in billion USDs (rounded to 2 decimal places), as logged by the International Monetary Fund (IMF). 
Since IMF releases this evaluation twice a year, this code will be used by the organization to extract the information as it is updated.

You can find the required data on this webpage (https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29).

The required information needs to be made accessible as a JSON file 'Countries_by_GDP.json' as well as 
a table 'Countries_by_GDP' in a database file 'World_Economies.db' with attributes 'Country' and 'GDP_USD_billion.'

Your boss wants you to demonstrate the success of this code by running a query on the database table to display only the entries with more 
than a 100 billion USD economy. Also, log the entire process of execution in a file named 'etl_project_log.txt'.

You must create a Python code 'etl_project_gdp.py' that performs all the required tasks.
'''

import pandas as pd
from bs4 import BeautifulSoup
import sqlite3 
import requests
import numpy as np
from datetime import datetime 

url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
json_file = 'Countries_by_GDP.json'
log_file = 'etl_project_log.txt'

def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 

'''
#get data from the url
html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')

#Need to get table info using find_all function.
tables = data.find_all('tbody')
rows = tables[2].find_all('tr')

for row in rows:
    #cells = row.find_all('td')
    #for cell in cells:
        #print(cell.text)
    col = row.find_all('td')
            if len(col)!=0:
                if col[0].find('a') is not None and '—' not in col[2]:
                    data_dict = {"Country": col[0].a.contents[0],
                                 "GDP_USD_millions": col[2].contents[0]}
                    df1 = pd.DataFrame(data_dict, index=[0])
                    df = pd.concat([df,df1], ignore_index=True)
    return df
'''        

def extract_transform(url):
    tables = pd.read_html(url)
    df = tables[3]
    #df = df[[0,2]] -- since its a multiindex due to merged cells, the columns of the df need to be re-numbered before we slice the df for just 2 cols.
    #print(df.columns)
    #using the shape[1] and range function we are giving a number to the col headers.
    df.columns = range(df.shape[1])
    #print(df.columns)
    df = df[[0,2]]
    df.columns = ['Country', 'IMF_GDP_USD_million']
    df = df.iloc[1:11, :]
    print (df)
    # Change the data type of the 'GDP (Million USD)' column to integer. Use astype() method.
    df[['IMF_GDP_USD_million']] = df[['IMF_GDP_USD_million']].astype(int)
    df[['IMF_GDP_USD_million']] = df[['IMF_GDP_USD_million']]/1000
    df[['IMF_GDP_USD_million']] = np.round(df[['IMF_GDP_USD_million']], 2)
    df.rename(columns={'IMF_GDP_USD_million' : 'IMF_GDP_USD_billion'})
    print (df)
    return df



def load(df):
    df.to_json(json_file)
    return df



def write_to_db(db_name):
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    
    log_progress("data collected from webscraping is also stored into a database")
    
    query_statement = f"SELECT * FROM {table_name} WHERE IMF_GDP_USD_million>=100"
    result = pd.read_sql(query_statement, conn)
    print(query_statement)
    print(result)
    conn.close()
    return 

df = extract_transform(url)
log_progress("Webscraping of data from webpage and required data transformation is also complete") 

df = load(df)
log_progress("data collected from webscraping is loaded into json file") 

write_to_db(db_name)
log_progress("sample query to read data from the database is complete")




   

           Country IMF_GDP_USD_million
1    United States            26854599
2            China            19373586
3            Japan             4409738
4          Germany             4308854
5            India             3736882
6   United Kingdom             3158938
7           France             2923489
8            Italy             2169745
9           Canada             2089672
10          Brazil             2081235
           Country  IMF_GDP_USD_million
1    United States             26854.60
2            China             19373.59
3            Japan              4409.74
4          Germany              4308.85
5            India              3736.88
6   United Kingdom              3158.94
7           France              2923.49
8            Italy              2169.74
9           Canada              2089.67
10          Brazil              2081.24
SELECT * FROM Countries_by_GDP WHERE IMF_GDP_USD_million>100
          Country  IMF_GDP_USD_million
0   United States             2

In [31]:
#Project Title: Acquiring and processing information on world's largest banks

# Importing the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3 
import requests
import numpy as np
from datetime import datetime 

url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
db_name = 'Banks.db'
table_name = 'Largest_banks'
log_file = 'code_log.txt'
csv_path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'
output_path = 'Largest_banks_data.csv'


def log_progress(message):
    '''Task_1_log_function: This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    time_stamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(time_stamp + ':' + message + '\n')

log_progress("Preliminaries complete. Initiating ETL process")


def extract(url):
    '''Task_2_extract: This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing.'''

    tables = pd.read_html(url)
    df = tables[1]
    df['Market cap (US$ billion)'] = df['Market cap (US$ billion)'].astype(float)
    return df

df = extract(url)
print (df)
log_progress ("Data extraction complete. Initiating Transformation process")

def transform(df, csv_path):
    '''Task_3_transform: This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

    csv_df = pd.read_csv(csv_path)
    #print(csv_df)
    ExchangeRate_dict = csv_df.set_index('Currency').to_dict()['Rate']
    #print (ExchangeRate_dict)
    df['MC_GBP_Billion'] = [np.round(x*ExchangeRate_dict['GBP'],2) for x in df['Market cap (US$ billion)']]
    df['MC_EUR_Billion'] = [np.round(x*ExchangeRate_dict['EUR'],2) for x in df['Market cap (US$ billion)']]
    df['MC_INR_Billion'] = [np.round(x*ExchangeRate_dict['INR'],2) for x in df['Market cap (US$ billion)']]
    print ("The market capitalization of the 5th largest bank in billion EUR is listed below:")
    print (df['MC_EUR_Billion'][4])
    return df 
    
transform(df, csv_path)
print(df)
log_progress ("Data transformation complete. Initiating Loading process")
    

def load_to_csv(df, output_path):
    ''' Task_4_CSV: This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path)

load_to_csv(df, output_path)
log_progress ("Data saved to CSV file")


conn = sqlite3.connect(db_name)
log_progress("SQL Connection initiated")

def load_to_db(df, conn, table_name):
    '''Task_5_load_to_DB: This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    
load_to_db(df, conn, table_name)
log_progress("Data loaded to Database as a table, Executing queries")


query_statement1 = "SELECT * FROM Largest_banks"
query_statement2 = "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
query_statement3 = "SELECT [Bank name] from Largest_banks LIMIT 5"

def run_query(query_statement, conn):
    '''Task_6_run_queries:This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing.'''
    
    result = pd.read_sql(query_statement, conn)
    print(query_statement)
    print(result)
    
run_query(query_statement1, conn)
run_query(query_statement2, conn)
run_query(query_statement3, conn)
log_progress("Process Complete")
conn.close()
log_progress("Server Connection closed")

'''
Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.
'''

   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91
5     6                              Wells Fargo                    155.87
6     7                        HSBC Holdings PLC                    148.90
7     8                           Morgan Stanley                    140.83
8     9                  China Construction Bank                    139.82
9    10                            Bank of China                    136.81
The market capitalization of the 5th largest bank in billion EUR is listed below:
146.86
   Rank                                Bank name  Market cap (US$ billion)  \
0     1 

'\nHere, you define the required entities and call the relevant\nfunctions in the correct order to complete the project. Note that this\nportion is not inside any function.\n'